In [116]:
#import ML frameworks
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import RidgeClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

rr = RidgeClassifier(alpha=1)

lreg = LogisticRegression(solver='liblinear')
svm = SVC()
knn = KNeighborsClassifier()
decTree = DecisionTreeClassifier()
rForest = RandomForestClassifier()
gaussNB = GaussianNB()
logReg = LogisticRegression()

split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, 
                                n_features_to_select=30, 
                                direction="forward",
                                cv=split,
                                n_jobs=1
                               )

In [117]:
# Import required libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

In [118]:
#import ML frameworks
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

rForest = RandomForestClassifier()

split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, 
                                n_features_to_select=30, 
                                direction="forward",
                                cv=split,
                                n_jobs=1
                               )

In [119]:
#adds "TARGET" column to data
def addTarget(group):
    group["TARGET"] = group["WIN"]
    return group

In [120]:
#runs given prediction model on given data
def fullTest(data, model, predictors, start=2, step=1):
    #list of dataframes where each df is the prediction for a season
    allPreds = []
    
    szns = sorted(data["YEAR"].unique())
    
    for sznIndex in range(start, len(szns), step):
        szn = szns[sznIndex]
        
        train = data[data["YEAR"] < szn]
        test = data[data["YEAR"] == szn]
        
        model.fit(data[predictors], data["TARGET"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        
        combined = pd.concat([test["TARGET"], preds], axis=1)
        combined.columns = ["ACTUAL", "PREDICTION"]
        
        allPreds.append(combined)
    
    return pd.concat(allPreds)

In [121]:
from sklearn.model_selection import cross_val_predict

#cross validates model, returns all predictions 
def crossVal(data, model, predictors, start=2, step=1):
    #list of dataframes where each df is the prediction for a season
    allPreds = []
    
    szns = sorted(data["YEAR"].unique())
    
    for sznIndex in range(start, len(szns), step):
        szn = szns[sznIndex]
        
        model.fit(data[predictors], data["TARGET"])
        cvPred = cross_val_predict(model, data[predictors], data["TARGET"], cv=3)
        
#         preds = model.predict(test[predictors])
#         preds = pd.Series(preds, index=test.index)
        
        combined = pd.concat([test["TARGET"], preds], axis=1)
        combined.columns = ["ACTUAL", "PREDICTION"]
        
        allPreds.append(combined)
    
    return pd.concat(allPreds)

In [122]:
#import and ready data for ML

import pandas as pd
filePath = "C:/Users/danna/Documents/GitHub/WBBTournamentPredictions/MachineLearning/LearningData/learningData8.csv"
df = pd.read_csv(filePath, index_col=0)
#add target column
df = addTarget(df)
#handles null values in target column
df["TARGET"][pd.isnull(df["TARGET"])] = 2
df["TARGET"] = df["TARGET"].astype(int, errors="ignore")
#select needed columns
removed_columns = ["YEAR", "WIN", "TEAM", "TEAM_OPP", "GAME_ID", "TARGET", "PTS", "PTS_OPP"]
selected_columns = df.columns[~df.columns.isin(removed_columns)]

C:\Users\danna\AppData\Local\Temp\ipykernel_34840\2799823127.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["TARGET"][pd.isnull(df["TARGET"])] = 2


In [123]:
#preprocess with Min Max Scaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [124]:
#handle unneeded columns
df[selected_columns][pd.isnull(df[selected_columns])] = 2
df[selected_columns] = df[selected_columns].astype(int, errors="ignore")

#fit the data 
sfs.fit(df[selected_columns], df["TARGET"])

#get predictor columns
predictors = list(selected_columns[sfs.get_support()])
# predictors

C:\Users\danna\AppData\Local\Temp\ipykernel_34840\3383206916.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_columns][pd.isnull(df[selected_columns])] = 2
C:\Users\danna\AppData\Local\Temp\ipykernel_34840\3383206916.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_columns][pd.isnull(df[selected_columns])] = 2


In [115]:
#create the variables to be used in prediction, X is predictor data, y is answers
X = df[predictors].values
y = df["TARGET"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

(666, 30)
(286, 30)


In [127]:
#create the variables to be used in prediction, X is predictor data, y is answers
X = df[predictors].values
y = df["TARGET"].values

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

clf = make_pipeline(StandardScaler(), SVC(gamma='auto', kernel='poly'))
clf.fit(X, y)

cvScore = cross_val_score(clf, df[predictors], df["TARGET"], cv=3)
cvScore

array([0.56918239, 0.58359621, 0.57097792])

In [ ]:
predictions = fullTest(df, rForest, predictors)

In [65]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(predictions["ACTUAL"], predictions["PREDICTION"])
# print(acc)

In [66]:
from sklearn.model_selection import cross_val_predict, cross_val_score
cvPred = cross_val_predict(rForest, df[predictors], df["TARGET"], cv=3)
cvScore = cross_val_score(rForest, df[predictors], df["TARGET"], cv=3)

In [67]:
#print out prediction results
print(predictions)
print(acc)
print(cvScore)

     ACTUAL  PREDICTION
240       0           0
241       1           1
242       0           0
243       1           1
244       0           1
..      ...         ...
947       1           1
948       1           1
949       0           1
950       0           0
951       1           1

[712 rows x 2 columns]
0.672752808988764
[0.68867925 0.66876972 0.57728707]


In [68]:
#define array of ML models to run through
models = [rr, lreg, svm, knn, decTree, rForest, gaussNB, logReg]

In [69]:
model = rForest
X = df[predictors]
y = df["TARGET"]
model.fit(X,y)

RandomForestClassifier()

In [70]:
#import clean test data, ready for ML

testPath = "C:/Users/danna/Documents/GitHub/WBBTournamentPredictions/MachineLearning/TestData/TestData8.csv"
ansPath = "C:/Users/danna/Documents/GitHub/WBBTournamentPredictions/MachineLearning/TestData/TestData8ANS.csv"
dfTest = pd.read_csv(testPath, index_col=0)
dfAns = pd.read_csv(ansPath, index_col=0)
# #add target column
# df = addTarget(df)
#handles null values in target column
# df["TARGET"][pd.isnull(df["TARGET"])] = 2
# df["TARGET"] = df["TARGET"].astype(int, errors="ignore")
#select needed columns
removed_columns = ["YEAR", "WIN", "TEAM", "TEAM_OPP", "GAME_ID", "TARGET", "PTS", "PTS_OPP"]
selected_columns = dfTest.columns[~dfTest.columns.isin(removed_columns)]


scaler = MinMaxScaler()
dfTest[selected_columns] = scaler.fit_transform(dfTest[selected_columns])

#handle unneeded columns
dfTest[selected_columns][pd.isnull(dfTest[selected_columns])] = 2
dfTest[selected_columns] = dfTest[selected_columns].astype(int, errors="ignore")

# #fit the data 
# sfs.fit(df[selected_columns], df["TARGET"])

#get predictor columns
# predictors = list(selected_columns[sfs.get_support()])
# predictors = selected_columns
# predictors

# predictions = fullTest(df, rForest, predictors)

C:\Users\danna\AppData\Local\Temp\ipykernel_34840\508634461.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTest[selected_columns][pd.isnull(dfTest[selected_columns])] = 2
C:\Users\danna\AppData\Local\Temp\ipykernel_34840\508634461.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTest[selected_columns][pd.isnull(dfTest[selected_columns])] = 2


In [71]:
#define predictor and answer variables (X,y respectively)
Xnew = dfTest[predictors]
yAns = dfAns

yNew = model.predict(Xnew)

#check accuracy of model
acc = accuracy_score(yAns, yNew)
print(acc)

0.5345744680851063


In [93]:
#format predictions in dataframe, change from int to bool values
pred = pd.DataFrame(data={'PRED': yNew}).astype(bool)
results = [pred, yAns]
results = pd.concat(results, axis=1)

display(results)

PRED    WIN
0     True  False
1     True   True
2     True  False
3    False   True
4     True  False
..     ...    ...
371   True   True
372  False   True
373   True  False
374   True  False
375   True   True

[376 rows x 2 columns]

In [58]:
#run specified models and check their accuracies

models = [rr, lreg, svm, knn, decTree, rForest, gaussNB, logReg]
accs = []
for model in models:
    X = df[predictors]
    y = df["TARGET"]
    model.fit(X,y)
    accs.append(accuracy_score(yAns, model.predict(Xnew)))

#print out maximum accuracy models
maxIndex = accs.index(max(accs))
topModels = []
for i in range(len(accs)):
    if(accs[i] == max(accs)):
        topModels.append(models[i])
print(topModels)

C:\Users\danna\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


[RidgeClassifier(alpha=1), LogisticRegression(solver='liblinear'), RandomForestClassifier(), LogisticRegression()]


0.5345744680851063